In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import keras

In [4]:
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt

In [5]:
data = pd.read_csv('datasets/r129c12s02/dataset_jan_r129c12s02.csv')

In [7]:
data = data.drop(['jobs', 'Unnamed: 0', 'timestamp'], axis = 1)

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [9]:
train_0, train_1 = train[train.label == 0], train[train.label == 1]
test_0, test_1 = test[test.label == 0], test[test.label == 1]

In [10]:
train_0.shape

(5550, 121)

In [11]:
from numpy import array
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.utils import plot_model


def DL_model(ishape = 120, full = True):
    
    visible = Input(shape=(ishape),name='input')
    encoder =Dense(100, activation='relu', name='encoder_0')(visible)
    layers = [80,60,40]
    names = [1,2,3]
    for i in range(len(layers)):
        encoder = Dense(layers[i], activation = 'relu',name='encoder_{}'.format(names[i]))(encoder)

    hidden = Dense(20,activation = 'relu', name='lattent')(encoder)

    # define reconstruct decoder
    decoder = Dense(40, activation='relu',name='decoder_0')(hidden)
    layers2 = [60,80,100]
    for i in range(len(layers2)):
        decoder = Dense(layers2[i], activation='relu',name='decoder_{}'.format(names[i]))(decoder)
    decoder = Dense(ishape, activation='sigmoid', name='autoencoder_output')(decoder)

    # define classifier
    classifier = Dense(20, activation='relu', name="hidden")(hidden)
    classifier = Dense(2, activation='softmax', name="classifier_output")(classifier)

    model = Model(inputs=visible, outputs=[decoder, classifier])

    model_dec = Model(inputs=visible, outputs=[decoder])
    model_cla = Model(inputs=visible, outputs=[classifier])
    
    if full:
        return model
    else:
        return model_dec

In [12]:
decoder = DL_model(120, False)
full = DL_model(120, True)

In [13]:
full.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={
        "autoencoder_output": keras.losses.MeanSquaredError(),
        "classifier_output": keras.losses.BinaryCrossentropy()
    },
    metrics={"autoencoder_output" : 'MeanSquaredError', "classifier_output": 'accuracy'}
)

In [14]:
decoder.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={
        "autoencoder_output": keras.losses.MeanSquaredError(),
    },
    metrics={"autoencoder_output" : 'MeanSquaredError'}
)

In [15]:
from sklearn.preprocessing import MinMaxScaler

# Training Autoencoder on normal data

In [16]:
X0_train = train_0.drop('label', axis = 1).values
scaler_dec = MinMaxScaler()
X0_train = scaler_dec.fit_transform(X0_train)
history_auto = decoder.fit(
    X0_train,
    {"autoencoder_output": X0_train},
    batch_size = 10,
    epochs=10)

Epoch 1/10
555/555 [==============================] - 1s 1ms/step - loss: 0.0137 - mean_squared_error: 0.0137: 0s - loss: 0.0188 - mean_squared
Epoch 2/10
555/555 [==============================] - 1s 1ms/step - loss: 0.0050 - mean_squared_error: 0.0050
Epoch 3/10
555/555 [==============================] - 1s 1ms/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 4/10
555/555 [==============================] - 1s 1ms/step - loss: 0.0030 - mean_squared_error: 0.0030
Epoch 5/10
555/555 [==============================] - 1s 1ms/step - loss: 0.0026 - mean_squared_error: 0.0026
Epoch 6/10
555/555 [==============================] - 1s 1ms/step - loss: 0.0024 - mean_squared_error: 0.0024
Epoch 7/10
555/555 [==============================] - 1s 1ms/step - loss: 0.0022 - mean_squared_error: 0.0022
Epoch 8/10
555/555 [==============================] - 1s 1ms/step - loss: 0.0021 - mean_squared_error: 0.0021
Epoch 9/10
555/555 [==============================] - 1s 1ms/step - loss: 0.0020 - mea

# Training classifier on all data

In [17]:
y_train = train.label.values
X_train = train.drop('label', axis = 1).values
scaler_cla = MinMaxScaler()
X0_train = scaler_cla.fit_transform(X0_train)
history_auto = full.fit(
    X_train,
    {"autoencoder_output": X_train, "classifier_output": y_train},
    batch_size = 10,
    epochs=10)

Epoch 1/10
556/556 [==============================] - 1s 1ms/step - loss: 130681226800340411923562496.0000 - autoencoder_output_loss: 130681226800340411923562496.0000 - classifier_output_loss: 7.6248 - autoencoder_output_mean_squared_error: 130681226800340411923562496.0000 - classifier_output_accuracy: 0.7079
Epoch 2/10
556/556 [==============================] - 1s 1ms/step - loss: 130681309810688743616544768.0000 - autoencoder_output_loss: 130681309810688743616544768.0000 - classifier_output_loss: 7.6248 - autoencoder_output_mean_squared_error: 130681309810688743616544768.0000 - classifier_output_accuracy: 0.7079
Epoch 3/10
556/556 [==============================] - 1s 1ms/step - loss: 130681263693828559342665728.0000 - autoencoder_output_loss: 130681263693828559342665728.0000 - classifier_output_loss: 7.6248 - autoencoder_output_mean_squared_error: 130681263693828559342665728.0000 - classifier_output_accuracy: 0.7079
Epoch 4/10
556/556 [==============================] - 1s 1ms/step -

# Combining the classifiers and evaluating the results

In [87]:
X0_test = test_0.drop('label', axis = 1).values
error_v = scaler_dec.fit_transform(X0_test) - decoder.predict(scaler_dec.fit_transform(X0_test))
#pd.DataFrame(error_v, columns = list(train.drop(['label'], axis = 1)))

In [90]:
error_v = pd.DataFrame(error_v, columns = list(train.drop(['label'], axis = 1))).ewm(span=data.shape[0]).mean()
#alpha = n_features/(datapoints+1)
error_v["sum"] = error_v.sum(axis=1)
error_v

,avg:Ambient_Temp,var:Ambient_Temp,avg:CMOS_Battery,var:CMOS_Battery,avg:CPU_1_DTS,var:CPU_1_DTS,avg:CPU_1_Temp,var:CPU_1_Temp,avg:CPU_2_DTS,var:CPU_2_DTS,...,var:pkts_out,avg:proc_run,var:proc_run,avg:proc_total,var:proc_total,avg:swap_free,var:swap_free,avg:swap_total,var:swap_total,sum
0,0.003149,-0.016751,0.000000,-2.817517e-08,0.040451,-0.000937,0.009706,-0.001725,-0.001446,-0.000898,...,-7.436348e-08,0.008117,-0.001161,0.046979,0.000069,-4.930515e-08,-4.149685e-08,-3.310027e-08,-4.355116e-08,-0.111612
1,0.025610,-0.016763,0.000032,-9.974891e-07,0.035425,-0.001051,0.008908,-0.001517,-0.000655,-0.001210,...,-1.570839e-06,0.002662,-0.001078,0.038291,0.000088,-7.315267e-07,-7.023107e-07,-3.732657e-07,-6.097453e-07,-0.024155
2,0.035449,-0.007686,0.001026,-2.676835e-05,0.032098,-0.001406,0.008578,-0.001666,0.000943,-0.001574,...,-3.218691e-05,0.002034,-0.003959,0.032865,0.000144,-4.630980e-05,-4.668091e-05,-3.169640e-05,-5.180388e-05,0.077591
3,0.042939,0.002756,0.001753,-4.562959e-05,0.030094,-0.001698,0.008646,-0.001828,0.003406,-0.001909,...,-5.462862e-05,0.001926,-0.006072,0.029253,0.000186,-7.976335e-05,-8.040674e-05,-5.470663e-05,-8.935346e-05,0.168920
4,0.047404,0.010699,0.002246,-5.845128e-05,0.027847,-0.001896,0.008016,-0.002020,0.005244,-0.002174,...,-6.992315e-05,0.001456,-0.007777,0.026751,0.000483,-1.025380e-04,-1.033446e-04,-7.041297e-05,-1.149143e-04,0.229344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,0.048034,0.027636,0.011229,-2.551782e-04,0.006270,-0.000458,-0.001494,-0.003227,0.009819,0.000005,...,8.729867e-03,-0.010496,0.004232,0.014161,0.007431,-2.767575e-04,-2.595184e-04,-2.548205e-04,-2.778852e-04,0.664798
1384,0.048036,0.027634,0.011231,-2.551595e-04,0.006271,-0.000457,-0.001493,-0.003226,0.009820,0.000005,...,8.728666e-03,-0.010494,0.004231,0.014158,0.007430,-2.767363e-04,-2.594972e-04,-2.547974e-04,-2.778585e-04,0.664819
1385,0.048038,0.027631,0.011233,-2.551408e-04,0.006272,-0.000456,-0.001492,-0.003225,0.009820,0.000005,...,8.727467e-03,-0.010492,0.004231,0.014156,0.007429,-2.767151e-04,-2.594761e-04,-2.547743e-04,-2.778318e-04,0.664840
1386,0.048040,0.027628,0.011235,-2.551222e-04,0.006273,-0.000455,-0.001492,-0.003223,0.009821,0.000006,...,8.726269e-03,-0.010490,0.004231,0.014154,0.007428,-2.766940e-04,-2.594550e-04,-2.547513e-04,-2.778050e-04,0.664861


In [108]:
def predict_au(data, thr = 0.6):
    data = data.drop('label', axis = 1).values
    error_v = scaler_dec.fit_transform(data) - decoder.predict(scaler_dec.fit_transform(data))
    error_v = pd.DataFrame(error_v, columns = list(train.drop(['label'], axis = 1))).ewm(span=data.shape[0]).mean()
    error_v["sum"] = error_v.sum(axis=1)
    error_v["label_pred"] = error_v["sum"].apply(lambda x: 1 if x >= thr else 0)
    return list(error_v["label_pred"])

In [129]:
def predict_cla(data):
    data = data.drop('label', axis = 1).values
    data = scaler_cla.fit_transform(data)
    prediction =  full.predict(data)[1]
    r = []
    for i in prediction:
        if i[0] > i[1]:
            r.append(0)
        else:
            r.append(1)
    return r

In [187]:
def predict_combined(data, thr = 0.95):
    pred_au = predict_au(data, thr)
    pred_cla = predict_cla(data)
    combined = [max(pred_au[i], pred_cla[i]) for i in range(len(pred_au))]
    return combined

In [188]:
from sklearn.metrics import classification_report

In [189]:
print(classification_report(list(test.label), predict_combined(test)))

              precision    recall  f1-score   support

           0       1.00      0.93      0.96      1388
           1       0.00      0.00      0.00         2

    accuracy                           0.93      1390
   macro avg       0.50      0.46      0.48      1390
weighted avg       1.00      0.93      0.96      1390



In [190]:
# comparion to random forest
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=10)
RF = RF.fit(train.drop('label', axis = 1).values, train.label)
pred = RF.predict(test.drop('label', axis = 1).values)
print(classification_report(list(test.label), pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1388
           1       0.50      0.50      0.50         2

    accuracy                           1.00      1390
   macro avg       0.75      0.75      0.75      1390
weighted avg       1.00      1.00      1.00      1390

